In [2]:
file = open('reviews.txt', 'r', encoding='utf-8')
corpus = file.read()

In [25]:
file2 = open('seed.txt', 'r', encoding='utf-8')
seed = file2.read()

In [30]:
with open('sentiment_dict.csv','r',encoding='utf-8') as file3:
    sentiment_dict = {word.split(',')[0] for word in file3.readlines()}

In [59]:
seed = set(seed.split('\n'))

In [3]:
corpus[:100]

'И пускай на меня не обижается наш прославленный защитник - франкофон «Монреаль Канадиенс» Maxime – я'

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import string
import re
import numpy as np

SEED = 1337

class TextNormalizer(TransformerMixin):
    def __init__(self, stop_words=stopwords.words('english'), 
                 stemmer=SnowballStemmer("english")):
        self.stop_words_ = stop_words
        self.stemmer_ = stemmer
        self.regex_ = re.compile('[%s]' % re.escape(string.punctuation))

    # by convention, must return self
    def fit(self, X, y):
        return self
    
    def _normalize(self, text):
        text = text.lower()
        text = self.regex_.sub(' ', text)
        text = [self.stemmer_.stem(word) for word in text.split() if word not in self.stop_words_]
        return " ".join(text)

    def transform(self, X):
        return np.array(list(map(lambda x: self._normalize(x), X)))

In [5]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
#from glove import Corpus, Glove


# convert words from tweet to vectors and average them over tweet
# or use weighted average according to idf
class Text2Vec(TransformerMixin):
    def __init__(self, vectorizer, use_idf=False):
        self.use_idf_ = use_idf
        self.w2v = vectorizer
        self.size_ = self.w2v.get_size()
        self.weights_ = None

    def fit(self, X, y=None):
        self.w2v.fit(X)
        
        if self.use_idf_:
            tfidf = TfidfVectorizer(analyzer=lambda x: x)
            tfidf.fit(X)
            # if a word was never seen - it must be at least as infrequent
            # as any of the known words - so the default idf is the max of known idf's
            max_idf = max(tfidf.idf_)
            self.weights_ = defaultdict(lambda: max_idf, [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        if self.w2v is None:
            raise Exception('model not fitted')
            
        if self.use_idf_:
            if self.weights_ is None:
                raise Exception('model not fitted')
                
            result = np.array([np.mean([self.w2v.transform(w) * self.weights_[w] for w in sentence.split() if self.w2v.has(w)]
                                       or [np.zeros(self.size_)], axis=0) for sentence in X])
        else:
            result = np.array([np.mean([self.w2v.transform(w) for w in sentence.split() if self.w2v.has(w)] 
                                       or [np.zeros(self.size_)], axis=0) for sentence in X])
        return result
    
    
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a word
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return self.w2v[X] if X in self.w2v else np.zeros(self.size_)
    

class WordEmbeddingFabric:
    @classmethod
    def create(cls, method, size, cbow, n_iter=300):
        if method == 'word2vec':
            return Word2VecWrapper(window=3, negative=5, size=size, iter=n_iter, is_cbow=cbow)
                    
        if method == 'glove':
            return GloveWrapper(window=3, learning_rate=0.05, size=size, epochs=n_iter, verbose=False)

In [6]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines()]
	return set(stopwords)

stop_words = load_stop_words("stoplist_russian.txt")
print(stop_words)

{'едва', 'этот', 'четыре', 'я', 'с', 'он', 'а', 'по-прежнему', 'пока', 'лишь', 'здесь', 'еще', 'даже', 'и', 'восемь', 'каждый', 'самый', 'она', 'год', 'ещё', 'два', 'рамка', 'над', 'примечательно', 'уже', 'стать', 'после', 'во', 'со', 'главный', 'что', 'по', 'первый', 'в', 'ли', 'на', 'свой'}


In [7]:
from pymystem3 import Mystem
m = Mystem()
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(input.lower()) if lemma.strip()]

def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

In [64]:
#remove_stop_words(lemmatize(corpus), stop_words)

In [8]:
import pandas as pd
corpus_lemm = pd.read_csv("reviews_score.csv")

In [20]:
corpus_lemm.head()

,Unnamed: 0,review,food,interior,service
0,0,и пускай на я не обижаться наш прославленный з...,8,8,8
1,1,- здравствовать. виа д’арженто! - добрый вечер...,9,7,10
2,2,"советовать вы увольнять ваш метродотель елена,...",9,9,1
3,3,отличный средне вековый интеръер. приятный обс...,8,10,9
4,4,ужинать в ресторан баден-баден 6 март . импоза...,10,8,8


In [9]:
text = '.'.join(corpus_lemm['review'])

In [18]:
sentences_list = [x.strip() for x in text.split('.')]

In [78]:
from gensim.models.word2vec import Word2Vec

# notice, that our dataset has window=2

w2v_cbow = Word2VecWrapper(window=2, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

In [10]:
import pickle
# pickle.dump(w2v_cbow, open("w2v_cbow", "wb"))
w2v_cbow = pickle.load(open("w2v_cbow", "rb"))

In [16]:
w2v_cbow.has('еда')

True

In [ ]:
columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']